In [ ]:
import sys
sys.path.append('/home/TomKerby/Research/lit_review/lit_review')
import utils
sys.path.append('/home/TomKerby/Research/lit_review/configs')
from new_llm_config import config
import rag_utils as rag

def handle_graph_update(paper_ids, cfg):
    emb_adapter = rag.get_embeddings(cfg)
    kg = utils.load_kg(cfg)
    
    result = kg.query(
        """
        MATCH (p:Paper) 
        WHERE p.paperId IN $paper_ids
        RETURN elementId(p) AS node_id, p.abstract AS abstract
        """,
        params={"paper_ids":paper_ids}
    )

    for record in result:
        if record["abstract"]:
            prepped_abstract = emb_adapter.prepare_query(record['abstract'])
            embedding = emb_adapter.embed_query(prepped_abstract)
            kg.query("""
                MATCH (p:Paper) WHERE elementId(p) = $node_id
                SET p.abstractEmbedding = $embedding
                RETURN elementId(p) AS node_id, p.abstract AS abstract
                """, params={"node_id":record["node_id"], "embedding":embedding}
            )

# paper_ids = ["04e541391e8dce14d099d00fb2c21dbbd8afe87f"]
# handle_graph_update(paper_ids, config)